In [ ]:
# Needs in terminal:
# $ quilc -S
# $ qvm -S

import numpy as np

from pyquil.api import get_benchmarker
from forest_benchmarking.rb import (add_sequences_to_dataframe,
                            add_survivals,
                            generate_simultaneous_rb_sequence,
                            rb_dataframe,
                            run_rb_measurement,
                            survivals_by_qubits)

%matplotlib inline

### You need a connection to the service for generating sequences

In [ ]:
bm = get_benchmarker()

# Simultaneous Sequence Generation

In [ ]:
seq_1q = generate_simultaneous_rb_sequence(bm=bm, subgraph=[(0,), (1,), (2,), (3,)], depth=5)
for idx, clifford_program in enumerate(seq_1q):
    print(f"Clifford {idx}:")
    print(clifford_program)

In [ ]:
seq_2q = generate_simultaneous_rb_sequence(bm=bm, subgraph=[(0, 5), (1, 6)], depth=5)
for idx, clifford_program in enumerate(seq_2q):
    print(f"Clifford {idx}:")
    print(clifford_program)

# Run RB experiments

## 1) Intialize and partially populate a DataFrame to summarize the experiment.

In [ ]:
df = rb_dataframe(rb_type="sim-2q",
                  subgraph=[(0, 5), (1, 6)],
                  depths=3 * 2 ** np.arange(3, dtype=np.uint8),
                  num_sequences=10)
df

## 2) Generate and add new RB sequences. This is slow!

In [ ]:
df = add_sequences_to_dataframe(df, bm)
df

## 3) Run the RB Sequences on a QuantumComputer

You can rerun this cell and get different results each time.

In [ ]:
from pyquil import get_qc
qc = get_qc("9q-square-noisy-qvm")

df = run_rb_measurement(df, qc, num_trials=100)
df 

# 4) Calculate survival statistics and add these to the DataFrame

In [ ]:
df = add_survivals(df)
df

# 5) Extract the survival data into arrays keyed to each measured qubit or qubit-pair

In [ ]:
subgraph = [(0, 5), (1, 6)]
depths, survivals, survival_errs = {}, {}, {}
for qubits in subgraph:
    depths[qubits], survivals[qubits], survival_errs[qubits] = survivals_by_qubits(df, qubits)

# 6) Fit the data

In [ ]:
from forest_benchmarking.analysis.fitting import make_figure
from forest_benchmarking.rb import fit_standard_rb

fit = fit_standard_rb(depths[(1, 6)], survivals[(1,6)], weights=1/survival_errs[(1, 6)])
fig, axs = make_figure(fit, xlabel="Sequence Length [Cliffords]", ylabel="Survival Probability")